In [1]:
import cantera as ct 
import numpy as np
import matplotlib.pylab as plt
import os 
import re
import csv
import pandas as pd

In [7]:
#scratchpad

blends = ['C2H5F_CH2FCH2F',
 'C2H5F_CH2FCHF2',
 'C2H5F_CH3CHF2',
 'C2H5F_CH3F',
 'CH2F2_CH3CF3',
 'CH2F2_CH3F',
 'CH2FCH2F_CH2F2',
 'CH2FCH2F_CH2FCHF2',
 'CH2FCH2F_CH3CHF2',
 'CH2FCH2F_CH3F',
 'CH2FCHF2_CH2F2',
 'CH2FCHF2_CH3F',
 'CH3CF3_C2H5F',
 'CH3CF3_CH2FCH2F',
 'CH3CF3_CH2FCHF2',
 'CH3CF3_CH3CHF2',
 'CH3CHF2_CH2F2',
 'CH3CHF2_CH2FCHF2',
 'CH3F_CH3CF3',
 'CH3F_CH3CHF2']

for blend in blends: 
    os.system(f'mv /work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{blend}/chemkin/dup_chem.cti /work/westgroup/nora/Code/projects/halogens/refrigerants/blends/blends_of_two_equal_parts/{blend}/chemkin/fixed_chem.cti')

# Gather Data

In [16]:
################ gather the data, where rxn_index is the index of the rmg reaction being changed to NIST kinetics
################ and flame_speeds is the flame speed of that model  in m/s
 

flame_speed_csvs = os.listdir('/scratch/westgroup/methane_flame_speed/one_flip/flame_speeds')
path_to_csvs = '/scratch/westgroup/methane_flame_speed/one_flip/flame_speeds/'

data = {}
for file in flame_speed_csvs: 
    #save the index
    raw_index = re.search('flame_speeds_(\d+).csv', file)
    index = int(str(raw_index.group(1)))
    
    #save the flame speed
    with open(os.path.join(path_to_csvs, file),'r') as f: 
        csvfile = csv.DictReader(f)
        for col in csvfile: 
            speed = col['1.0']
#             if not speed: 
#                 print(file)
#                 break
            data[index] = float(speed)

sorted_data = sorted(data.items())
sorted_data


[(0, 0.23917745696295445),
 (1, 0.25184862740439945),
 (2, 0.2460063433912924),
 (3, 0.24968047278611374),
 (5, 0.24883000665904473),
 (6, 0.24883000665904514),
 (7, 0.24883318379925512),
 (8, 0.24883000665904478),
 (9, 0.24883000665904506),
 (10, 0.24867515327358958),
 (11, 0.24865619107024928),
 (12, 0.2495557850316756),
 (13, 0.248807714575589),
 (14, 0.24869317028702176),
 (15, 0.2501243774334967),
 (16, 0.24861760305797234),
 (17, 0.24706135009118518),
 (18, 0.21373028417873557),
 (19, 0.24876769106396454),
 (20, 0.2519222208486636),
 (21, 0.2486403127376914),
 (22, 0.24917326845940066),
 (23, 0.24880931760537559),
 (24, 0.2440317209029915),
 (25, 0.24875534234543356),
 (26, 0.24882795172070218),
 (27, 0.24755363381922738),
 (28, 0.24883001149267953),
 (29, 0.2518919628136004),
 (30, 0.24873563994846717),
 (31, 0.2488316349337136),
 (32, 0.24883003234056086),
 (33, 0.25709402694514477),
 (34, 0.25470550518242685),
 (35, 0.24883631795581226),
 (36, 0.24872719064440005),
 (37, 0.248

# Plot

In [18]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
rxn_indexes = [rxn for rxn, spd in sorted_data]
flame_speed = [spd for rxn, spd in sorted_data]
ax.bar(rxn_indexes, flame_speed)
plt.show()

In [12]:
################# which have the fastest flame speed ############################################



df = pd.DataFrame((zip(rxn_indexes, flame_speed)), columns = ['Reaction Index', 'Speed'])
# print(df)
fastest = df.nlargest(10, 'Speed')
fastest 

[96, 91, 33, 99, 34, 20, 29]


,Reaction Index,Speed
259,260,0.327574
95,96,0.267016
260,261,0.263601
90,91,0.258358
32,33,0.257094
98,99,0.256058
33,34,0.254706
383,384,0.251970
19,20,0.251922
28,29,0.251892


In [ ]:
import rmgpy.chemkin

full_path = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/'


RMG_chemkin_path = full_path + 'no_halogens.inp'
RMG_dictionary_path = full_path + 'no_halogens_dict.txt'
RMG_transport_path = full_path + 'no_halogens_tran.dat'
RMG_cti_path = full_path + 'no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)

In [ ]:
fastest_list = fastest['Reaction Index'].to_list()
for i in fastest_list: 
    print(RMG_reaction_list[i])

# Try on the Entire RMG Model (without anything deleted)


In [24]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
import numpy as np
import subprocess

import csv

import mix_thermokinetics

import scipy
import copy
import os

In [25]:
#top ten fastest flame speed reactions 
rxns_to_use_handpicked = [96, 91, 33, 99, 34, 20, 29] #only the ones that are in RMG and NIST, this way it won't delete the one's that don't have a NIST equivalent

In [26]:
#load in NIST and RMG models. Both have no halogens

# Load the models
# load David's 2-BTP model, minus the halogens
# https://github.com/comocheng/halogen_models/tree/main/combustion_symposium_paper/supporting_material

full_path = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/difftool/'


RMG_chemkin_path = full_path + 'no_halogens.inp'
RMG_dictionary_path = full_path + 'no_halogens_dict.txt'
RMG_transport_path = full_path + 'no_halogens_tran.dat'
RMG_cti_path = full_path + 'no_halogens.cti'

RMG_species_list, RMG_reaction_list = rmgpy.chemkin.load_chemkin_file(RMG_chemkin_path, dictionary_path=RMG_dictionary_path, transport_path=RMG_transport_path)
RMG_gas = ct.Solution(RMG_cti_path)


# Load NIST model without halogens
NIST_cti_path = full_path + 'NIST_no_halogens/no_halogens.cti'
NIST_chemkin_path = full_path + 'NIST_no_halogens/no_halogens.inp'
NIST_dictionary_path = full_path + 'NIST_no_halogens/no_halogens_dict.txt'
NIST_transport_path = full_path + 'NIST_no_halogens/no_halogens_tran.dat'

NIST_gas = ct.Solution(NIST_cti_path)
NIST_dict = rmgpy.chemkin.load_species_dictionary(NIST_dictionary_path)
NIST_species_list, NIST_reaction_list = rmgpy.chemkin.load_chemkin_file(NIST_chemkin_path, dictionary_path=NIST_dictionary_path, transport_path=NIST_transport_path)



**** WARNING ****
For species CH2OCH, discontinuity in cp/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  8.39347
	Value computed using high-temperature polynomial: 9.1801


**** WARNING ****
For species CH2OCH, discontinuity in h/RT detected at Tmid = 500
	Value computed using low-temperature polynomial:  42.1991
	Value computed using high-temperature polynomial: 41.9615


**** WARNING ****
For species CH2OCH, discontinuity in s/R detected at Tmid = 500
	Value computed using low-temperature polynomial:  33.7069
	Value computed using high-temperature polynomial: 33.5121


**** WARNING ****
For species C4H5-2, discontinuity in h/RT detected at Tmid = 1000
	Value computed using low-temperature polynomial:  47.6524
	Value computed using high-temperature polynomial: 48.4362


**** WARNING ****
For species C4H5-2, discontinuity in s/R detected at Tmid = 1000
	Value computed using low-temperature polynomial:  52.4292
	Value computed using high-temperature polyno

In [27]:
# get the mapping between RMG and NIST models
# Species Diff
common_species = []
RMG2NIST_mapping = {}
NIST2RMG_mapping = {}
for i, rmg_sp in enumerate(RMG_species_list):
    for j, nist_sp in enumerate(NIST_species_list):
        if rmg_sp.is_isomorphic(nist_sp):
            RMG2NIST_mapping[i] = j
            NIST2RMG_mapping[j] = i
            common_species.append([rmg_sp, nist_sp])
            break

# Reaction Diff
common_reactions = []
RMG2NIST_rxn_mapping = {}
NIST2RMG_rxn_mapping = {}
for i, rmg_rxn in enumerate(RMG_reaction_list):
    for j, nist_rxn in enumerate(NIST_reaction_list):
        if rmg_rxn.is_isomorphic(nist_rxn):
            RMG2NIST_rxn_mapping[i] = j
            NIST2RMG_rxn_mapping[j] = i
            common_reactions.append([rmg_rxn, nist_rxn])
            break
print(f'{len(common_species)} common species')
print(f'{len(common_reactions)} common reactions')

45 common species
240 common reactions


In [28]:
RMG_indices_that_are_in_NIST = []
for i in rxns_to_use_handpicked: 
    if i in RMG2NIST_rxn_mapping.keys(): 
        RMG_indices_that_are_in_NIST.append(i)

print(RMG_indices_that_are_in_NIST)
print(rxns_to_use_handpicked)
        
        
if RMG_indices_that_are_in_NIST == rxns_to_use_handpicked: 
    print('ooo la la')

[96, 91, 33, 99, 34, 20, 29]
[96, 91, 33, 99, 34, 20, 29]
ooo la la


In [29]:
# Convert the NIST species in the reactions to RMG species, but keep the NIST kinetics
def NIST2RMG(nist_reaction):
    rmg_reaction = copy.deepcopy(nist_reaction)
    reactants = []
    for reactant in nist_reaction.reactants:
        try:
            NIST_species_index = NIST_species_list.index(reactant)
            reactants.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if reactant in RMG_species_list:
                reactants.append(reactant)
        
    rmg_reaction.reactants = reactants
    
    products = []
    for product in nist_reaction.products:
        try:
            NIST_species_index = NIST_species_list.index(product)
            products.append(RMG_species_list[NIST2RMG_mapping[NIST_species_index]])
        except ValueError:
            if product in RMG_species_list:
                products.append(product)
    rmg_reaction.products = products
    
    return rmg_reaction

In [30]:
def RMG2NIST(RMG_reaction):
    # takes in the RMG_reaction object to convert
    RMG_index = RMG_reaction_list.index(RMG_reaction)
    if RMG_index not in RMG2NIST_rxn_mapping.keys():
        # this reaction does not exist in NIST, so it will be deleted. return None
        return
    NIST_index = RMG2NIST_rxn_mapping[RMG_index]
    NIST_reaction = NIST_reaction_list[NIST_index]
    
    # convert the NIST model species in the NIST_reaction to RMG model species
    return NIST2RMG(NIST_reaction)

In [35]:
handpicked_dir = 'hand_picked'
os.makedirs(divide_dir, exist_ok=True)

In [32]:
# convert the indicated reactions to use the NIST kinetics
new_reaction_list = []
deleted_duplicates = []
for i in range(0, len(RMG_reaction_list)):
    if i in rxns_to_use_handpicked:
        new_reaction = RMG2NIST(RMG_reaction_list[i])
        if new_reaction:
            new_reaction_list.append(new_reaction)
        elif RMG_reaction_list[i].duplicate:
            deleted_duplicates.append(RMG_reaction_list[i])
    else:
        new_reaction_list.append(RMG_reaction_list[i])

In [33]:
# get rid of duplicates
for i, rxn in enumerate(new_reaction_list):
    if rxn.duplicate:
        duplicate_still_exists = False
        for j, rxn2 in enumerate(new_reaction_list):
            if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                duplicate_still_exists = True
                break
        if not duplicate_still_exists:
            rxn.duplicate = False
            
# mark reactions that are duplicates
for i, rxn in enumerate(new_reaction_list):
    if not rxn.duplicate:
        duplicate = False
        for j, rxn2 in enumerate(new_reaction_list):
            if rxn.is_isomorphic(rxn2) and rxn != rxn2:
                duplicate = True
                break
        if duplicate:
            rxn.duplicate = True

In [37]:
# save the model
chemkin_file = os.path.join(handpicked_dir, 'chem.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, RMG_species_list, new_reaction_list, verbose=True, check_for_duplicates=True)
# subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={RMG_transport_path}', f'--output={divide_dir}/chem.cti'])

In [ ]:
################### delete files past a certain index if the calculations are garbage #######################
for file in flame_speed_csvs: 
    raw_index = re.search('flame_speeds_(\d+).csv', file)
    index = int(str(raw_index.group(1)))
    if index > 316:
        string_ = f'rm {os.path.join(path_to_csvs, file)}'
        os.system(string_)
      